In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import MT5ForConditionalGeneration, AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate import meteor_score
from datasets import load_dataset

import time
import nltk
import numpy as np

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jinhyunpark/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# english to X is only possible for T5
from datasets import load_dataset, load_from_disk

# ds_de_en = load_dataset("wmt/wmt14", "de-en")
# ds_fr_en = load_dataset("wmt/wmt15", "fr-en")
# ds_ro_en = load_dataset("wmt/wmt16", "ro-en")

# ds_de_en.save_to_disk("../wmt14_de_en")
# ds_fr_en.save_to_disk("../wmt15_fr_en")
# ds_ro_en.save_to_disk("../wmt16_ro_en")

ds_de_en = load_from_disk("../wmt14_de_en")
ds_fr_en = load_from_disk("../wmt15_fr_en")
ds_ro_en = load_from_disk("../wmt16_ro_en")

In [6]:
t5_tokenizer_small = T5Tokenizer.from_pretrained('t5-small')
t5_model_small = T5ForConditionalGeneration.from_pretrained('t5-small')
print("Done with small")

t5_tokenizer_base = T5Tokenizer.from_pretrained('t5-base')
t5_model_base = T5ForConditionalGeneration.from_pretrained('t5-base')
print("Done with base")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Done with small
Done with base


In [7]:
input_text_t5 = "translate English to French: I like to eat pizza"

t5_input_ids_small = t5_tokenizer_small.encode(input_text_t5, return_tensors='pt')
t5_outputs_small = t5_model_small.generate(t5_input_ids_small)
t5_output_text = t5_tokenizer_small.decode(t5_outputs_small[0], skip_special_tokens=True)
print(t5_outputs_small)

/Users/jinhyunpark/miniconda3/lib/python3.12/site-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [7]:
input_text_t5 = "translate English to French: I like to eat pizza"

t5_input_ids_base = t5_tokenizer_base.encode(input_text_t5, return_tensors='pt')
t5_outputs_base = t5_model_base.generate(t5_input_ids_base)
t5_output_text = t5_tokenizer_base.decode(t5_outputs_base[0], skip_special_tokens=True)
print(t5_outputs_base)

: 